In [17]:
import os, sys, time, errno
import numpy as np
import scipy as sp
import mvpa2.suite as mvpa2

add_delay = 6
TR = 2.3
offset = 5

In [18]:
def read_and_preprocess_picture_data(data_file,mask_file,log_file,nuisance_file,brightness_file,contrast_file):
    
    ds = mvpa2.fmri_dataset(data_file,mask=mask_file,add_fa={'cluster':mask_file})

    log = np.recfromcsv(log_file,delimiter=',')

    with open(nuisance_file) as f:
        nuisance = f.readlines()
    reg1 = []
    reg2 = []
    for nuisance_line in nuisance:
        a = nuisance_line.split(' ')
        if len(a) == 2:
            reg1.append(float(a[0]))
            reg2.append(float(a[1]))

    ds.sa['reg1']=reg1
    ds.sa['reg2']=reg2
    ds.sa['timepoint'] = TR * np.arange(len(ds))
    ds.sa['chunks'] = [0 for i in range(len(ds))]
    
    ## Regress away nuisance regressors (white-matter and background noise)
    ds = ds[offset:]

    opt_regs = ['reg1','reg2']
    
    mvpa2.poly_detrend(ds,polyord=1,opt_regs=opt_regs)
    mvpa2.zscore(ds,chunks_attr=None)

    ## Select all the scans where the pictures are shown (onset+6s)
    scans = np.round((log.onset + add_delay) / TR).astype(int) - offset

    # First
    ds1 = ds[scans].copy(deep=True)
    ds2 = ds[scans+1].copy(deep=True)

    ds1.sa['chunks'] = log.session
    ds1.sa['category'] = log.category
    
    ds1.sa['brightness'] = np.log(log.brightness)
    #ds1.sa['contrast'] = np.log(log.contrast)
        
    ds1.samples = (ds1.samples + ds2.samples) / 2
    mvpa2.poly_detrend(ds1,polyord=0,opt_regs=['brightness'])
    
    ds1 = ds1[np.argsort(log.picture)]
    
    sample_mean = np.mean(ds1,axis=1)
    for i,s in enumerate(sample_mean):
        ds1.samples[i,:] -= s

    return ds1

In [26]:
def read_and_preprocess_brand_data(data_file,mask_file,log_file,nuisance_file,brightness_file,contrast_file):
    
    ds = mvpa2.fmri_dataset(data_file,mask=mask_file,add_fa={'cluster':mask_file})

    log = np.recfromcsv(log_file,delimiter=',')

    with open(nuisance_file) as f:
        nuisance = f.readlines()
    reg1 = []
    reg2 = []
    for nuisance_line in nuisance:
        a = nuisance_line.split(' ')
        if len(a) == 2:
            reg1.append(float(a[0]))
            reg2.append(float(a[1]))

    ds.sa['reg1']=reg1
    ds.sa['reg2']=reg2
    ds.sa['timepoint'] = TR * np.arange(len(ds))
    ds.sa['chunks'] = [0 for i in range(len(ds))]

    ## Regress away nuisance regressors (white-matter and background noise)
    ds = ds[offset:]

    opt_regs = ['reg1','reg2']
    
    mvpa2.poly_detrend(ds,polyord=1,opt_regs=opt_regs)
    mvpa2.zscore(ds,chunks_attr=None)

    ## Select all the scans where the pictures are shown (onset+6s)
    scans = np.round(log.onset / TR).astype(int) - offset
    timepoint = np.round(log.onset / TR).astype(int) * TR - log.onset

    ds_t = []
    for i in range(10):
        ds_e = ds[scans+i]

        ds_e.sa['chunks'] = log.session
        ds_e.sa['brand'] = log.brand
        ds_e.sa['brightness'] = np.log(log.brightness)
        #ds_e.sa['contrast'] = np.log(log.contrast)
        ds_e.sa['timepoint'] = timepoint + (TR*i)
        ds_e.sa['scan'] = [i for x in range(len(ds_e))]

        mvpa2.poly_detrend(ds_e,polyord=0,opt_regs=['brightness'])
        
        ds_t.append(ds_e)
          
    ds_t = mvpa2.vstack(ds_t)

    return ds_t

In [27]:
method = 'normalized_common'
method0 = method.split('_')[0]
method1 = method.split('_')[1]

#roi_mask = ['picture_thresh1','picture_thresh2','picture_thresh5','picture_thresh2_no_occip']
roi_mask = ['picture_thresh2']
#roi_mask = ['picture_p001','picture_p0005','picture_p0001']
#roi_mask = ['picture_0.5_clust_10','picture_1_clust_10','picture_2_clust_10','picture_5_clust_10']
#roi_mask = ['picture_1_no_occip_clust_10']
#roi_mask = ['picture_2_clust_10_no_occip']

In [28]:
for r in roi_mask:
    print r
    
    vsize = []
    subj_ds = []
    
    for subj in range(4,42):
        mask = []
        mask_all = []

        ## For each subject    
        start_time = time.time()

        data_file = '../nifti/'+method0+'/picture/S'+str(subj)+'_picture_'+method0+'.nii'
        
        if method1 == "spm":
            mask_file = '../roi/'+method0+'/common_mask/spm/picture2.nii'            
        elif method1 == "common":
            if method0 == "native":
                mask_file = '../roi/'+method0+'/common_mask/'+r+'/S'+str(subj)+'/w'+r+'.nii'
            else:
                mask_file = '../roi/'+method0+'/common_mask/'+r+'.nii'
        else:
            mask_file = '../roi/'+method0+'/'+r+'/S'+str(subj)+'.nii'
            
        log_file = '../log/S'+str(subj)+'_picture.csv'
        nuisance_file = '../nuisance/S'+str(subj)+'_picture_nuisance.txt'
        brightness_file = '../nuisance/S'+str(subj)+'_picture_brightness.txt'
        contrast_file = '../nuisance/S'+str(subj)+'_picture_contrast.txt'
        
        ds = read_and_preprocess_picture_data(data_file,mask_file,log_file,nuisance_file,brightness_file,contrast_file)

        elapsed_time = time.time() - start_time

        print "Subject "+str(subj)+" "+str(ds.shape)+" Elapsed time: "+'{:1.2f}'.format(elapsed_time)+'s'
        vsize.append(ds.shape[1])
        subj_ds.append(ds)

    subj_ds = mvpa2.hstack(subj_ds)
    mvpa2.save(subj_ds,'../output/'+method+'/'+method+'_'+r+'.h5py')

picture_thresh2
Subject 4 (112, 3173) Elapsed time: 2.65s
Subject 5 (112, 3173) Elapsed time: 2.59s
Subject 6 (112, 3173) Elapsed time: 2.62s
Subject 7 (112, 3173) Elapsed time: 2.65s
Subject 8 (112, 3173) Elapsed time: 2.65s
Subject 9 (112, 3173) Elapsed time: 2.65s
Subject 10 (112, 3173) Elapsed time: 2.53s
Subject 11 (112, 3173) Elapsed time: 3.01s
Subject 12 (112, 3173) Elapsed time: 2.60s
Subject 13 (112, 3173) Elapsed time: 3.12s
Subject 14 (112, 3173) Elapsed time: 2.62s
Subject 15 (112, 3173) Elapsed time: 2.85s
Subject 16 (112, 3173) Elapsed time: 2.59s
Subject 17 (112, 3173) Elapsed time: 2.60s
Subject 18 (112, 3173) Elapsed time: 2.78s
Subject 19 (112, 3173) Elapsed time: 2.49s
Subject 20 (112, 3173) Elapsed time: 2.34s
Subject 21 (112, 3173) Elapsed time: 2.60s
Subject 22 (112, 3173) Elapsed time: 2.40s
Subject 23 (112, 3173) Elapsed time: 2.30s
Subject 24 (112, 3173) Elapsed time: 2.88s
Subject 25 (112, 3173) Elapsed time: 3.00s
Subject 26 (112, 3173) Elapsed time: 2.58s
S

In [29]:
for r in roi_mask:
    print r

    output_path = '../output/'+method+'/'+r.replace("picture","brand")
    try:
        os.makedirs(output_path)
    except OSError as exception:
        if exception.errno != errno.EEXIST:
            raise
    
    vsize = []
    
    for subj in range(4,42):
        mask = []
        mask_all = []

        ## For each subject    
        start_time = time.time()

        data_file = '../nifti/'+method0+'/brand/S'+str(subj)+'_brand_'+method0+'.nii'
        
        if method1 == "spm":
            mask_file = '../roi/'+method0+'/common_mask/spm/picture2.nii'            
        elif method1 == "common":
            if method0 == "native":
                mask_file = '../roi/'+method0+'/common_mask/'+r+'/S'+str(subj)+'/w'+r+'.nii'
            else:
                mask_file = '../roi/'+method0+'/common_mask/'+r+'.nii'
        else:
            mask_file = '../roi/'+method0+'/'+r+'/S'+str(subj)+'.nii'
            
        log_file = '../log/S'+str(subj)+'_brand.csv'
        nuisance_file = '../nuisance/S'+str(subj)+'_brand_nuisance.txt'
        brightness_file = '../nuisance/S'+str(subj)+'_brand_brightness.txt'
        contrast_file = '../nuisance/S'+str(subj)+'_brand_contrast.txt'
        
        ds = read_and_preprocess_brand_data(data_file,mask_file,log_file,nuisance_file,brightness_file,contrast_file)

        elapsed_time = time.time() - start_time

        print "Subject "+str(subj)+" "+str(ds.shape)+" Elapsed time: "+'{:1.2f}'.format(elapsed_time)+'s'
        vsize.append(ds.shape[1])
        
        ds.sa['subj'] = [subj for i in range(len(ds))]
        mvpa2.save(ds,output_path+'/S'+str(subj)+'.h5py')

picture_thresh2
Subject 4 (840, 3173) Elapsed time: 1.92s
Subject 5 (840, 3173) Elapsed time: 2.16s
Subject 6 (840, 3173) Elapsed time: 2.48s
Subject 7 (840, 3173) Elapsed time: 2.57s
Subject 8 (840, 3173) Elapsed time: 2.28s
Subject 9 (840, 3173) Elapsed time: 2.66s
Subject 10 (840, 3173) Elapsed time: 2.85s
Subject 11 (840, 3173) Elapsed time: 2.27s
Subject 12 (840, 3173) Elapsed time: 2.57s
Subject 13 (840, 3173) Elapsed time: 2.67s
Subject 14 (840, 3173) Elapsed time: 2.67s
Subject 15 (840, 3173) Elapsed time: 2.31s
Subject 16 (840, 3173) Elapsed time: 2.10s
Subject 17 (840, 3173) Elapsed time: 2.42s
Subject 18 (840, 3173) Elapsed time: 2.50s
Subject 19 (840, 3173) Elapsed time: 2.10s
Subject 20 (840, 3173) Elapsed time: 2.77s
Subject 21 (840, 3173) Elapsed time: 2.18s
Subject 22 (840, 3173) Elapsed time: 2.45s
Subject 23 (840, 3173) Elapsed time: 1.92s
Subject 24 (840, 3173) Elapsed time: 2.59s
Subject 25 (840, 3173) Elapsed time: 2.64s
Subject 26 (840, 3173) Elapsed time: 2.23s
S